In [1]:
import numpy as np
from itertools import permutations
import gzip
import pandas as pd
from qiskit import (QuantumCircuit,execute,Aer)
from qiskit.visualization import plot_histogram
from qiskit import*
from docplex.mp.model import Model                                                                                                      
import docplex
# from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
# import qiskit.optimization.applications.ising.docplex
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import time
from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.algorithms import QAOA

from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit.aqua.components.optimizers import COBYLA

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
from qiskit import IBMQ

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute
from qiskit.tools.jupyter import *
from qiskit.providers.ibmq import least_busy
from qiskit.optimization.applications.ising.tsp import TspData
import qiskit.optimization.applications.ising.tsp as tsp
from qiskit.optimization.applications.ising.common import sample_most_likely
import qiskit.optimization.applications.ising.docplex
import qiskit.optimization.applications.ising.docplex as docplex
from qiskit.circuit.library import TwoLocal
from qiskit.circuit.library import RealAmplitudes

In [2]:
from qiskit import IBMQ
# Load account from disk
IBMQ.load_account()
IBMQ.providers()

In [18]:
def conversion1():
    G = []
    qw = open("Matrices1a.txt" , "r")
    wq = qw.read().split()
    for i in range (len(wq)):
        wq[i] = int(float(wq[i]))
    G.append(np.reshape(wq,(3,3)))

    p = [4,5,6,7,8,9,10,11]
    q = [i**(2) for i in p ]
    m = 0
    v = open("Matrices.txt"  , "r")
    w = v.read().split()
    for i in range (len(w)):
        w[i] = int(float(w[i]))
    for i in range (len(q)):
        G.append(np.reshape(w[m:m+q[i]] , (p[i] , p[i])))
        m = m + q[i]
    return G
A =  conversion1()[0:3]
sol = [1285 , 1318 , 1612 , 2333,2014,1636,2213,2376][0:3]

In [5]:
def conversion2(result):
    data = sorted(result['eigenstate'].items(), key=lambda item: item[1])[::-1]
    for i in range(len(data)):
        a = tsp.tsp_feasible(data[i][0])
        arr = 'Infeasible'
        if a == True:
            b = str(data[i][0])
            arr = [b , data[i][1]]
            break
    return arr

In [20]:
def optimal(a,b,c,f,u):
    qw = open("optimal.txt" , "r")
    wq = qw.read().split()
    t = []
    for i in wq:
        if i != ',':
            q = len(i)
            t.append(float(i[0:q-1]))

    v = np.array(t[0:a]) #3
    r = np.array(t[a:a+b]) #4 
    o = np.array(t[a+b : a+b+c]) #5
    d = np.array(t[a+b+c:a+b+c+f]) #6
    z = np.array(t[a+b+c+f:a+b+c+f+u]) #7
    return [v,r,o,d,z]
R = optimal(54,96,100,216,294)[0:3]

In [7]:
def VQE1(n,s,A,vf,c):
    
    x = TspData('tmp',len(A),np.zeros((3,3)),A)
    qubitOp ,offset = tsp.get_operator(x)
    
    seed = 10598
    spsa = SPSA(maxiter=n)
    
    if vf == 'vf1':
        ry = RealAmplitudes(qubitOp.num_qubits, entanglement='linear')
    elif vf == 'vf2':
        ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')

    vqe =  VQE(qubitOp, ry, spsa,include_custom=True,initial_point = c )
    my_provider = IBMQ.get_provider('ibm-q-wits')
    device = my_provider.get_backend('ibmq_cambridge')

    quantum_instance = QuantumInstance(device, seed_simulator=seed, seed_transpiler=seed,shots = s,
                                       skip_qobj_validation = False
                                     )
    
    result = vqe.run(quantum_instance)
    answer = conversion2(result)
    if answer == 'Infeasible':
        solution = -1
    else:
        binarry = [int(p) for p in answer[0]]
        route = tsp.get_tsp_solution(binarry)
        solution = tsp.tsp_value(route,A)
        
    return  solution ,result['optimizer_time'],answer[1]

In [ ]:
def device(A,R):
    
    for j in range(len(A)):
        new_file2 = open("TSP_cambridge_.csv" + str(len(A[j])) , "w")
        for i in range(30):
            new_file2.write(str(VQE1(1,1024,A[j],'vf2',R[j])) + "\n")
        print(i)
        new_file2.close()
        
    return 'Done'

In [10]:
c3 = "TSP_cambridge_3.csv"
c4 =  "TSP_cambridge_4.csv"
c5 = "TSP_cambridge_5.csv"

In [11]:
def Analysis(string , optimal ):
    
    r = open(string , "r")
    o = r.read().split()
    Res = np.zeros((30,2)).astype(object)

    p = 0
    q = 0
    
    for i in range(3,93,3):
        g = o[p:i]
        p = p + 3
        Res[q,0] = float(g[0][1:-1])
        Res[q,1] = float(g[1][0:-1])
        q = q + 1

    h = 0
    n = 0
    #SR95
    for l in Res[:,0]:
        if l <= optimal*1.05:
            if  l != -1:
                h = h + 1
    #SR99
    for k in Res[:,0]:
        if k <= optimal*1.01:
            if k != -1:
                n = n + 1
    #Average time
    #Minimum Distance
    if min(Res[:,0]) == -1:
        y = np.sort(Res[:,0])
        if y[-1] == -1:
            P = 'Infeasible'
        else:
            for i in y:
                if i != -1:
                    P = i
                    break
    else:
        P = min(Res[:,0])
    return "Optimal Distance:",P,"SR99:",round((n/30)*100,2) , "SR95:",round((h/30)*100,2) , "AVERAGE TIME", np.mean(Res[:,1]) 


In [12]:
Analysis(c3 , sol[0] )

('Optimal Distance:',
 1285.0,
 'SR99:',
 100.0,
 'SR95:',
 100.0,
 'AVERAGE TIME',
 423.8472289641698)

In [13]:
Analysis(c4 , sol[1] )

('Optimal Distance:',
 1318.0,
 'SR99:',
 16.67,
 'SR95:',
 16.67,
 'AVERAGE TIME',
 365.3496110359828)

In [14]:
Analysis(c5 , sol[2] )

('Optimal Distance:',
 'Infeasible',
 'SR99:',
 0.0,
 'SR95:',
 0.0,
 'AVERAGE TIME',
 1135.7747276465097)

In [15]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.15.1',
 'qiskit-aer': '0.6.1',
 'qiskit-ignis': '0.4.0',
 'qiskit-ibmq-provider': '0.8.0',
 'qiskit-aqua': '0.7.5',
 'qiskit': '0.20.0'}